In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

english_to_french = [
 ("I am cold", "J'ai froid"),
 ("You are tired", "Tu es fatigué"),
 ("He is hungry", "Il a faim"),
 ("She is happy", "Elle est heureuse"),
 ("We are friends", "Nous sommes amis"),
 ("They are students", "Ils sont étudiants"),
 ("The cat is sleeping", "Le chat dort"),
 ("The sun is shining", "Le soleil brille"),
 ("We love music", "Nous aimons la musique"),
 ("She speaks French fluently", "Elle parle français couramment"),
 ("He enjoys reading books", "Il aime lire des livres"),
 ("They play soccer every weekend", "Ils jouent au football chaque week-end"),
 ("The movie starts at 7 PM", "Le film commence à 19 heures"),
 ("She wears a red dress", "Elle porte une robe rouge"),
 ("We cook dinner together", "Nous cuisinons le dîner ensemble"),
 ("He drives a blue car", "Il conduit une voiture bleue"),
 ("They visit museums often", "Ils visitent souvent des musées"),
 ("The restaurant serves delicious food", "Le restaurant sert une délicieuse cuisine"),
 ("She studies mathematics at university", "Elle étudie les mathématiques àl'université"),
 ("We watch movies on Fridays", "Nous regardons des films le vendredi"),
 ("He listens to music while jogging", "Il écoute de la musique en faisant du jogging"),
 ("They travel around the world", "Ils voyagent autour du monde"),
 ("The book is on the table", "Le livre est sur la table"),
 ("She dances gracefully", "Elle danse avec grâce"),
 ("We celebrate birthdays with cake", "Nous célébrons les anniversaires avec ungâteau"),
 ("He works hard every day", "Il travaille dur tous les jours"),
 ("They speak different languages", "Ils parlent différentes langues"),
 ("The flowers bloom in spring", "Les fleurs fleurissent au printemps"),
 ("She writes poetry in her free time", "Elle écrit de la poésie pendant son tempslibre"),
 ("We learn something new every day", "Nous apprenons quelque chose de nouveauchaque jour"),
 ("The dog barks loudly", "Le chien aboie bruyamment"),
 ("He sings beautifully", "Il chante magnifiquement"),
 ("They swim in the pool", "Ils nagent dans la piscine"),
 ("The birds chirp in the morning", "Les oiseaux gazouillent le matin"),
 ("She teaches English at school", "Elle enseigne l'anglais à l'école"),
 ("We eat breakfast together", "Nous prenons le petit déjeuner ensemble"),
 ("He paints landscapes", "Il peint des paysages"),
 ("They laugh at the joke", "Ils rient de la blague"),
 ("The clock ticks loudly", "L'horloge tic-tac bruyamment"),
 ("She runs in the park", "Elle court dans le parc"),
 ("We travel by train", "Nous voyageons en train"),
 ("He writes a letter", "Il écrit une lettre"),
 ("They read books at the library", "Ils lisent des livres à la bibliothèque"),
 ("The baby cries", "Le bébé pleure"),
 ("She studies hard for exams", "Elle étudie dur pour les examens"),
 ("We plant flowers in the garden", "Nous plantons des fleurs dans le jardin"),
 ("He fixes the car", "Il répare la voiture"),
 ("They drink coffee in the morning", "Ils boivent du café le matin"),
 ("The sun sets in the evening", "Le soleil se couche le soir"),
 ("She dances at the party", "Elle danse à la fête"),
 ("We play music at the concert", "Nous jouons de la musique au concert"),
 ("He cooks dinner for his family", "Il cuisine le dîner pour sa famille"),
 ("They study French grammar", "Ils étudient la grammaire française"),
 ("The rain falls gently", "La pluie tombe doucement"),
 ("She sings a song", "Elle chante une chanson"),
 ("We watch a movie together", "Nous regardons un film ensemble"),
 ("He sleeps deeply", "Il dort profondément"),
 ("They travel to Paris", "Ils voyagent à Paris"),
 ("The children play in the park", "Les enfants jouent dans le parc"),
 ("She walks along the beach", "Elle se promène le long de la plage"),
 ("We talk on the phone", "Nous parlons au téléphone"),
 ("He waits for the bus", "Il attend le bus"),
 ("They visit the Eiffel Tower", "Ils visitent la tour Eiffel"),
 ("The stars twinkle at night", "Les étoiles scintillent la nuit"),
 ("She dreams of flying", "Elle rêve de voler"),
 ("We work in the office", "Nous travaillons au bureau"),
 ("He studies history", "Il étudie l'histoire"),
 ("They listen to the radio", "Ils écoutent la radio"),
 ("The wind blows gently", "Le vent souffle doucement"),
 ("She swims in the ocean", "Elle nage dans l'océan"),
 ("We dance at the wedding", "Nous dansons au mariage"),
 ("He climbs the mountain", "Il gravit la montagne"),
 ("They hike in the forest", "Ils font de la randonnée dans la forêt"),
 ("The cat meows loudly", "Le chat miaule bruyamment"),
 ("She paints a picture", "Elle peint un tableau"),
 ("We build a sandcastle", "Nous construisons un château de sable"),
 ("He sings in the choir", "Il chante dans le chœur")
]


SOS_token = 0  
EOS_token = 1 
max_length = 12

def build_vocab(sentences):
    vocab = set()
    for pair in sentences:
        english_sentence, french_sentence = pair
        for word in english_sentence.split():
            vocab.add(word)
        for word in french_sentence.split():
            vocab.add(word)
    return vocab

english_vocab = build_vocab(english_to_french)
french_vocab = english_vocab  # Lets say French and English have the same vocabulary

char_to_index_english = {"SOS": SOS_token, "EOS": EOS_token, **{char: i+2 for i, char in enumerate(sorted(list(english_vocab)))}}
index_to_char_english = {i: char for char, i in char_to_index_english.items()}

char_to_index_french = {"SOS": SOS_token, "EOS": EOS_token, **{char: i+2 for i, char in enumerate(sorted(list(french_vocab)))}}
index_to_char_french = {i: char for char, i in char_to_index_french.items()}

class CustomDataset(Dataset):
    def __init__(self, dataset, char_to_index_english, char_to_index_french):
        self.dataset = dataset
        self.char_to_index_english = char_to_index_english
        self.char_to_index_french = char_to_index_french

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        english_sentence, french_sentence = self.dataset[idx]
        english_tensor = torch.tensor([self.char_to_index_english[word] for word in english_sentence.split()] + [EOS_token], dtype=torch.long)
        french_tensor = torch.tensor([self.char_to_index_french[word] for word in french_sentence.split()] + [EOS_token], dtype=torch.long)
        return english_tensor, french_tensor

custom_dataset = CustomDataset(english_to_french, char_to_index_english, char_to_index_french)
dataloader = DataLoader(custom_dataset, batch_size=1, shuffle=True)

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size), torch.zeros(1, 1, self.hidden_size))

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        output = torch.log_softmax(self.out(output[0]), dim=1)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size), torch.zeros(1, 1, self.hidden_size))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

learning_rate = 0.01
criterion = nn.NLLLoss()
encoder = EncoderRNN(input_size=len(char_to_index_english), hidden_size=256).to(device)
decoder = DecoderRNN(hidden_size=256, output_size=len(char_to_index_french)).to(device)
encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

# Training loop
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=12):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()

        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
        if decoder_input.item() == EOS_token:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

n_epochs = 100

for epoch in range(n_epochs):
    total_loss = 0
    for input_tensor, target_tensor in dataloader:
        input_tensor = input_tensor[0].to(device)
        target_tensor = target_tensor[0].to(device)

        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        total_loss += loss

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Training Loss: {total_loss / len(dataloader)}')

def evaluate(encoder, decoder, dataloader, criterion, n_samples=4):
    encoder.eval()
    decoder.eval()

    total_loss = 0
    total_sentences = 0
    correct_predictions = 0

    with torch.no_grad():
        for i, (input_tensor, target_tensor) in enumerate(dataloader):
            input_tensor = input_tensor[0].to(device)
            target_tensor = target_tensor[0].to(device)

            encoder_hidden = encoder.initHidden()
            input_length = input_tensor.size(0)
            target_length = target_tensor.size(0)

            encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

            loss = 0

            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)
                encoder_outputs[ei] = encoder_output[0, 0]

            decoder_input = torch.tensor([[SOS_token]], device=device)
            decoder_hidden = encoder_hidden

            predicted_indices = []

            for di in range(target_length):
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                topv, topi = decoder_output.topk(1)
                predicted_indices.append(topi.item())
                decoder_input = topi.squeeze().detach()

                loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
                if decoder_input.item() == EOS_token:
                    break

            total_loss += loss.item() / target_length
            total_sentences += 1

            if predicted_indices == target_tensor.tolist():
                correct_predictions += 1

            if i < n_samples:
                predicted_sentence = ' '.join([index_to_char_french[index] for index in predicted_indices if index not in (SOS_token, EOS_token)])
                target_sentence = ' '.join([index_to_char_french[index.item()] for index in target_tensor if index.item() not in (SOS_token, EOS_token)])
                input_sentence = ' '.join([index_to_char_english[index.item()] for index in input_tensor if index.item() not in (SOS_token, EOS_token)])

                print(f'Input String: {input_sentence}, Target String: {target_sentence}, Predicted String: {predicted_sentence}')

        average_loss = total_loss / len(dataloader)
        accuracy = correct_predictions / total_sentences
        print(f'Evaluation Loss: {average_loss}, Accuracy: {accuracy}')

# Perform evaluation
evaluate(encoder, decoder, dataloader, criterion)


Epoch 0, Training Loss: 4.242859690117115
Epoch 10, Training Loss: 2.947662201200015
Epoch 20, Training Loss: 2.276210608818536
Epoch 30, Training Loss: 1.6448998703068982
Epoch 40, Training Loss: 0.8712005125741521
Epoch 50, Training Loss: 0.23682633690387045
Epoch 60, Training Loss: 0.10465145424373913
Epoch 70, Training Loss: 0.05374086870284111
Epoch 80, Training Loss: 0.03621804902871056
Epoch 90, Training Loss: 0.027552139458030162
Input String: They listen to the radio, Target String: Ils écoutent la radio, Predicted String: Ils écoutent la radio
Input String: We eat breakfast together, Target String: Nous prenons le petit déjeuner ensemble, Predicted String: Nous prenons le petit déjeuner ensemble
Input String: The flowers bloom in spring, Target String: Les fleurs fleurissent au printemps, Predicted String: Les fleurs fleurissent au printemps
Input String: We plant flowers in the garden, Target String: Nous plantons des fleurs dans le jardin, Predicted String: Nous plantons de

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

english_to_french = [
 ("I am cold", "J'ai froid"),
 ("You are tired", "Tu es fatigué"),
 ("He is hungry", "Il a faim"),
 ("She is happy", "Elle est heureuse"),
 ("We are friends", "Nous sommes amis"),
 ("They are students", "Ils sont étudiants"),
 ("The cat is sleeping", "Le chat dort"),
 ("The sun is shining", "Le soleil brille"),
 ("We love music", "Nous aimons la musique"),
 ("She speaks French fluently", "Elle parle français couramment"),
 ("He enjoys reading books", "Il aime lire des livres"),
 ("They play soccer every weekend", "Ils jouent au football chaque week-end"),
 ("The movie starts at 7 PM", "Le film commence à 19 heures"),
 ("She wears a red dress", "Elle porte une robe rouge"),
 ("We cook dinner together", "Nous cuisinons le dîner ensemble"),
 ("He drives a blue car", "Il conduit une voiture bleue"),
 ("They visit museums often", "Ils visitent souvent des musées"),
 ("The restaurant serves delicious food", "Le restaurant sert une délicieuse cuisine"),
 ("She studies mathematics at university", "Elle étudie les mathématiques àl'université"),
 ("We watch movies on Fridays", "Nous regardons des films le vendredi"),
 ("He listens to music while jogging", "Il écoute de la musique en faisant du jogging"),
 ("They travel around the world", "Ils voyagent autour du monde"),
 ("The book is on the table", "Le livre est sur la table"),
 ("She dances gracefully", "Elle danse avec grâce"),
 ("We celebrate birthdays with cake", "Nous célébrons les anniversaires avec ungâteau"),
 ("He works hard every day", "Il travaille dur tous les jours"),
 ("They speak different languages", "Ils parlent différentes langues"),
 ("The flowers bloom in spring", "Les fleurs fleurissent au printemps"),
 ("She writes poetry in her free time", "Elle écrit de la poésie pendant son tempslibre"),
 ("We learn something new every day", "Nous apprenons quelque chose de nouveauchaque jour"),
 ("The dog barks loudly", "Le chien aboie bruyamment"),
 ("He sings beautifully", "Il chante magnifiquement"),
 ("They swim in the pool", "Ils nagent dans la piscine"),
 ("The birds chirp in the morning", "Les oiseaux gazouillent le matin"),
 ("She teaches English at school", "Elle enseigne l'anglais à l'école"),
 ("We eat breakfast together", "Nous prenons le petit déjeuner ensemble"),
 ("He paints landscapes", "Il peint des paysages"),
 ("They laugh at the joke", "Ils rient de la blague"),
 ("The clock ticks loudly", "L'horloge tic-tac bruyamment"),
 ("She runs in the park", "Elle court dans le parc"),
 ("We travel by train", "Nous voyageons en train"),
 ("He writes a letter", "Il écrit une lettre"),
 ("They read books at the library", "Ils lisent des livres à la bibliothèque"),
 ("The baby cries", "Le bébé pleure"),
 ("She studies hard for exams", "Elle étudie dur pour les examens"),
 ("We plant flowers in the garden", "Nous plantons des fleurs dans le jardin"),
 ("He fixes the car", "Il répare la voiture"),
 ("They drink coffee in the morning", "Ils boivent du café le matin"),
 ("The sun sets in the evening", "Le soleil se couche le soir"),
 ("She dances at the party", "Elle danse à la fête"),
 ("We play music at the concert", "Nous jouons de la musique au concert"),
 ("He cooks dinner for his family", "Il cuisine le dîner pour sa famille"),
 ("They study French grammar", "Ils étudient la grammaire française"),
 ("The rain falls gently", "La pluie tombe doucement"),
 ("She sings a song", "Elle chante une chanson"),
 ("We watch a movie together", "Nous regardons un film ensemble"),
 ("He sleeps deeply", "Il dort profondément"),
 ("They travel to Paris", "Ils voyagent à Paris"),
 ("The children play in the park", "Les enfants jouent dans le parc"),
 ("She walks along the beach", "Elle se promène le long de la plage"),
 ("We talk on the phone", "Nous parlons au téléphone"),
 ("He waits for the bus", "Il attend le bus"),
 ("They visit the Eiffel Tower", "Ils visitent la tour Eiffel"),
 ("The stars twinkle at night", "Les étoiles scintillent la nuit"),
 ("She dreams of flying", "Elle rêve de voler"),
 ("We work in the office", "Nous travaillons au bureau"),
 ("He studies history", "Il étudie l'histoire"),
 ("They listen to the radio", "Ils écoutent la radio"),
 ("The wind blows gently", "Le vent souffle doucement"),
 ("She swims in the ocean", "Elle nage dans l'océan"),
 ("We dance at the wedding", "Nous dansons au mariage"),
 ("He climbs the mountain", "Il gravit la montagne"),
 ("They hike in the forest", "Ils font de la randonnée dans la forêt"),
 ("The cat meows loudly", "Le chat miaule bruyamment"),
 ("She paints a picture", "Elle peint un tableau"),
 ("We build a sandcastle", "Nous construisons un château de sable"),
 ("He sings in the choir", "Il chante dans le chœur")
]


START_TOKEN = 0
END_TOKEN = 1
MAX_LENGTH = 12


def build_vocab(sentences):
    vocab = set()
    for pair in sentences:
        english_sentence, french_sentence = pair
        for word in english_sentence.split():
            vocab.add(word)
        for word in french_sentence.split():
            vocab.add(word)
    return vocab

english_vocab = build_vocab(english_to_french)
french_vocab = english_vocab

char_to_index_english = {"START": START_TOKEN, "END": END_TOKEN, **{char: i+2 for i, char in enumerate(sorted(list(english_vocab)))}}
index_to_char_english = {i: char for char, i in char_to_index_english.items()}

char_to_index_french = {"START": START_TOKEN, "END": END_TOKEN, **{char: i+2 for i, char in enumerate(sorted(list(french_vocab)))}}
index_to_char_french = {i: char for char, i in char_to_index_french.items()}

class CustomDataset(Dataset):
    def __init__(self, dataset, char_to_index_english, char_to_index_french):
        self.dataset = dataset
        self.char_to_index_english = char_to_index_english
        self.char_to_index_french = char_to_index_french

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        english_sentence, french_sentence = self.dataset[idx]
        english_tensor = torch.tensor([self.char_to_index_english[word] for word in english_sentence.split()] + [END_TOKEN], dtype=torch.long)
        french_tensor = torch.tensor([self.char_to_index_french[word] for word in french_sentence.split()] + [END_TOKEN], dtype=torch.long)
        return english_tensor, french_tensor

custom_dataset = CustomDataset(english_to_french, char_to_index_english, char_to_index_french)
dataloader = DataLoader(custom_dataset, batch_size=1, shuffle=True)

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size), torch.zeros(1, 1, self.hidden_size))

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        output = torch.log_softmax(self.out(output[0]), dim=1)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size), torch.zeros(1, 1, self.hidden_size))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


learning_rate = 0.01
criterion = nn.NLLLoss()
encoder = EncoderRNN(input_size=len(char_to_index_english), hidden_size=256).to(device)
decoder = DecoderRNN(hidden_size=256, output_size=len(char_to_index_french)).to(device)
encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

# Training loop
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=12):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[START_TOKEN]], device=device)
    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()

        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
        if decoder_input.item() == END_TOKEN:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

n_epochs = 100

for epoch in range(n_epochs):
    total_loss = 0
    for input_tensor, target_tensor in dataloader:
        input_tensor = input_tensor[0].to(device)
        target_tensor = target_tensor[0].to(device)

        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        total_loss += loss

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Training Loss: {total_loss / len(dataloader)}')

def evaluate(encoder, decoder, dataloader, criterion, n_samples=4):
    encoder.eval()
    decoder.eval()

    total_loss = 0
    total_sentences = 0
    correct_predictions = 0

    with torch.no_grad():
        for i, (input_tensor, target_tensor) in enumerate(dataloader):
            input_tensor = input_tensor[0].to(device)
            target_tensor = target_tensor[0].to(device)

            encoder_hidden = encoder.initHidden()
            input_length = input_tensor.size(0)
            target_length = target_tensor.size(0)

            encoder_outputs = torch.zeros(MAX_LENGTH, encoder.hidden_size, device=device)

            loss = 0

            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)
                encoder_outputs[ei] = encoder_output[0, 0]

            decoder_input = torch.tensor([[START_TOKEN]], device=device)
            decoder_hidden = encoder_hidden

            predicted_indices = []

            for di in range(target_length):
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                topv, topi = decoder_output.topk(1)
                predicted_indices.append(topi.item())
                decoder_input = topi.squeeze().detach()

                loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
                if decoder_input.item() == END_TOKEN:
                    break

            total_loss += loss.item() / target_length
            total_sentences += 1

            if predicted_indices == target_tensor.tolist():
                correct_predictions += 1

            if i < n_samples:
                predicted_sentence = ' '.join([index_to_char_french[index] for index in predicted_indices if index not in (START_TOKEN, END_TOKEN)])
                target_sentence = ' '.join([index_to_char_french[index.item()] for index in target_tensor if index.item() not in (START_TOKEN, END_TOKEN)])
                input_sentence = ' '.join([index_to_char_english[index.item()] for index in input_tensor if index.item() not in (START_TOKEN, END_TOKEN)])

                print(f'Input String: {input_sentence}, Target String: {target_sentence}, Predicted String: {predicted_sentence}')

        average_loss = total_loss / len(dataloader)
        accuracy = correct_predictions / total_sentences
        print(f'Evaluation Loss: {average_loss}, Accuracy: {accuracy}')

# Perform evaluation
evaluate(encoder, decoder, dataloader, criterion)

Epoch 0, Training Loss: 4.125024635809963
Epoch 10, Training Loss: 2.985630531194062
Epoch 20, Training Loss: 2.4106955486728934
Epoch 30, Training Loss: 1.7148085326171374
Epoch 40, Training Loss: 0.8286409782505705
Epoch 50, Training Loss: 0.2700547598604934
Epoch 60, Training Loss: 0.1130979223452912
Epoch 70, Training Loss: 0.06337273663646707
Epoch 80, Training Loss: 0.038034056361140336
Epoch 90, Training Loss: 0.027509726617708757
Input String: They speak different languages, Target String: Ils parlent différentes langues, Predicted String: Ils parlent différentes langues
Input String: He paints landscapes, Target String: Il peint des paysages, Predicted String: Il peint des paysages
Input String: He writes a letter, Target String: Il écrit une lettre, Predicted String: Il écrit une lettre
Input String: The movie starts at 7 PM, Target String: Le film commence à 19 heures, Predicted String: Le film commence à 19 heures
Evaluation Loss: 0.021795336485801405, Accuracy: 1.0


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset


english_to_french = [
 ("I am cold", "J'ai froid"),
 ("You are tired", "Tu es fatigué"),
 ("He is hungry", "Il a faim"),
 ("She is happy", "Elle est heureuse"),
 ("We are friends", "Nous sommes amis"),
 ("They are students", "Ils sont étudiants"),
 ("The cat is sleeping", "Le chat dort"),
 ("The sun is shining", "Le soleil brille"),
 ("We love music", "Nous aimons la musique"),
 ("She speaks French fluently", "Elle parle français couramment"),
 ("He enjoys reading books", "Il aime lire des livres"),
 ("They play soccer every weekend", "Ils jouent au football chaque week-end"),
 ("The movie starts at 7 PM", "Le film commence à 19 heures"),
 ("She wears a red dress", "Elle porte une robe rouge"),
 ("We cook dinner together", "Nous cuisinons le dîner ensemble"),
 ("He drives a blue car", "Il conduit une voiture bleue"),
 ("They visit museums often", "Ils visitent souvent des musées"),
 ("The restaurant serves delicious food", "Le restaurant sert une délicieuse cuisine"),
 ("She studies mathematics at university", "Elle étudie les mathématiques àl'université"),
 ("We watch movies on Fridays", "Nous regardons des films le vendredi"),
 ("He listens to music while jogging", "Il écoute de la musique en faisant du jogging"),
 ("They travel around the world", "Ils voyagent autour du monde"),
 ("The book is on the table", "Le livre est sur la table"),
 ("She dances gracefully", "Elle danse avec grâce"),
 ("We celebrate birthdays with cake", "Nous célébrons les anniversaires avec ungâteau"),
 ("He works hard every day", "Il travaille dur tous les jours"),
 ("They speak different languages", "Ils parlent différentes langues"),
 ("The flowers bloom in spring", "Les fleurs fleurissent au printemps"),
 ("She writes poetry in her free time", "Elle écrit de la poésie pendant son tempslibre"),
 ("We learn something new every day", "Nous apprenons quelque chose de nouveauchaque jour"),
 ("The dog barks loudly", "Le chien aboie bruyamment"),
 ("He sings beautifully", "Il chante magnifiquement"),
 ("They swim in the pool", "Ils nagent dans la piscine"),
 ("The birds chirp in the morning", "Les oiseaux gazouillent le matin"),
 ("She teaches English at school", "Elle enseigne l'anglais à l'école"),
 ("We eat breakfast together", "Nous prenons le petit déjeuner ensemble"),
 ("He paints landscapes", "Il peint des paysages"),
 ("They laugh at the joke", "Ils rient de la blague"),
 ("The clock ticks loudly", "L'horloge tic-tac bruyamment"),
 ("She runs in the park", "Elle court dans le parc"),
 ("We travel by train", "Nous voyageons en train"),
 ("He writes a letter", "Il écrit une lettre"),
 ("They read books at the library", "Ils lisent des livres à la bibliothèque"),
 ("The baby cries", "Le bébé pleure"),
 ("She studies hard for exams", "Elle étudie dur pour les examens"),
 ("We plant flowers in the garden", "Nous plantons des fleurs dans le jardin"),
 ("He fixes the car", "Il répare la voiture"),
 ("They drink coffee in the morning", "Ils boivent du café le matin"),
 ("The sun sets in the evening", "Le soleil se couche le soir"),
 ("She dances at the party", "Elle danse à la fête"),
 ("We play music at the concert", "Nous jouons de la musique au concert"),
 ("He cooks dinner for his family", "Il cuisine le dîner pour sa famille"),
 ("They study French grammar", "Ils étudient la grammaire française"),
 ("The rain falls gently", "La pluie tombe doucement"),
 ("She sings a song", "Elle chante une chanson"),
 ("We watch a movie together", "Nous regardons un film ensemble"),
 ("He sleeps deeply", "Il dort profondément"),
 ("They travel to Paris", "Ils voyagent à Paris"),
 ("The children play in the park", "Les enfants jouent dans le parc"),
 ("She walks along the beach", "Elle se promène le long de la plage"),
 ("We talk on the phone", "Nous parlons au téléphone"),
 ("He waits for the bus", "Il attend le bus"),
 ("They visit the Eiffel Tower", "Ils visitent la tour Eiffel"),
 ("The stars twinkle at night", "Les étoiles scintillent la nuit"),
 ("She dreams of flying", "Elle rêve de voler"),
 ("We work in the office", "Nous travaillons au bureau"),
 ("He studies history", "Il étudie l'histoire"),
 ("They listen to the radio", "Ils écoutent la radio"),
 ("The wind blows gently", "Le vent souffle doucement"),
 ("She swims in the ocean", "Elle nage dans l'océan"),
 ("We dance at the wedding", "Nous dansons au mariage"),
 ("He climbs the mountain", "Il gravit la montagne"),
 ("They hike in the forest", "Ils font de la randonnée dans la forêt"),
 ("The cat meows loudly", "Le chat miaule bruyamment"),
 ("She paints a picture", "Elle peint un tableau"),
 ("We build a sandcastle", "Nous construisons un château de sable"),
 ("He sings in the choir", "Il chante dans le chœur")
]


french_to_english = [(french, english) for (english, french) in english_to_french]

SOS_token = 0 
EOS_token = 1  
max_length = 12

def build_vocab(sentences):
    vocab = set()
    for pair in sentences:
        english_sentence, french_sentence = pair
        for word in english_sentence.split():
            vocab.add(word)
        for word in french_sentence.split():
            vocab.add(word)
    return vocab

english_vocab = build_vocab(english_to_french)
french_vocab = english_vocab

char_to_index_eng = {"SOS": SOS_token, "EOS": EOS_token, **{char: i+2 for i, char in enumerate(sorted(list(english_vocab)))}}
index_to_char_eng = {i: char for char, i in char_to_index_eng.items()}

char_to_index_fr = {"SOS": SOS_token, "EOS": EOS_token, **{char: i+2 for i, char in enumerate(sorted(list(french_vocab)))}}
index_to_char_fr = {i: char for char, i in char_to_index_fr.items()}

class CustomDataset(Dataset):
    def __init__(self, dataset, char_to_index_eng, char_to_index_fr):
        self.dataset = dataset
        self.char_to_index_eng = char_to_index_eng
        self.char_to_index_fr = char_to_index_fr

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        eng_sentence, fr_sentence = self.dataset[idx]
        eng_tensor = torch.tensor([self.char_to_index_eng[word] for word in eng_sentence.split()] + [EOS_token], dtype=torch.long)
        fr_tensor = torch.tensor([self.char_to_index_fr[word] for word in fr_sentence.split()] + [EOS_token], dtype=torch.long)
        return eng_tensor, fr_tensor

custom_dataset = CustomDataset(french_to_english, char_to_index_fr, char_to_index_eng)
dataloader = DataLoader(custom_dataset, batch_size=1, shuffle=True)

# Initializing encoder and decoder
input_size_eng = len(char_to_index_eng)
input_size_fr = len(char_to_index_fr)
hidden_size = 256  

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, max_length=12, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = torch.softmax(
            self.attn(torch.cat((embedded[0], hidden[0][0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = torch.relu(output)
        output, hidden = self.lstm(output, hidden)

        output = torch.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


learning_rate = 0.01
criterion = nn.NLLLoss()
encoder = EncoderRNN(input_size=input_size_eng, hidden_size=hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size=hidden_size, output_size=input_size_fr).to(device)
encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

# Training loop
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=12):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()

        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
        if decoder_input.item() == EOS_token:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

n_epochs = 100

for epoch in range(n_epochs):
    total_loss = 0
    for input_tensor_eng, target_tensor_fr in dataloader:
        input_tensor_eng = input_tensor_eng[0].to(device)
        target_tensor_fr = target_tensor_fr[0].to(device)

        loss = train(input_tensor_eng, target_tensor_fr, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        total_loss += loss

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Training Loss: {total_loss / len(dataloader)}')

def evaluate(encoder, decoder, dataloader, criterion, n_samples=4):
    encoder.eval()
    decoder.eval()

    total_loss = 0
    total_sentences = 0
    correct_predictions = 0

    with torch.no_grad():
        for i, (input_tensor_eng, target_tensor_fr) in enumerate(dataloader):
            input_tensor_eng = input_tensor_eng[0].to(device)
            target_tensor_fr = target_tensor_fr[0].to(device)

            encoder_hidden = encoder.initHidden()
            input_length = input_tensor_eng.size(0)
            target_length = target_tensor_fr.size(0)

            encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

            loss = 0

            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(input_tensor_eng[ei].unsqueeze(0), encoder_hidden)
                encoder_outputs[ei] = encoder_output[0, 0]

            decoder_input = torch.tensor([[SOS_token]], device=device)
            decoder_hidden = encoder_hidden

            predicted_indices = []

            for di in range(target_length):
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)
                topv, topi = decoder_output.topk(1)
                predicted_indices.append(topi.item())
                decoder_input = topi.squeeze().detach()

                loss += criterion(decoder_output, target_tensor_fr[di].unsqueeze(0))
                if decoder_input.item() == EOS_token:
                    break

            total_loss += loss.item() / target_length
            total_sentences += 1

            if predicted_indices == target_tensor_fr.tolist():
                correct_predictions += 1

            if i < n_samples:
                predicted_sentence = ' '.join([index_to_char_fr[index] for index in predicted_indices if index not in (SOS_token, EOS_token)])
                target_sentence = ' '.join([index_to_char_fr[index.item()] for index in target_tensor_fr if index.item() not in (SOS_token, EOS_token)])
                input_sentence = ' '.join([index_to_char_eng[index.item()] for index in input_tensor_eng if index.item() not in (SOS_token, EOS_token)])

                print(f'Input String: {input_sentence}, Target String: {target_sentence}, Predicted String: {predicted_sentence}')

        average_loss = total_loss / len(dataloader)
        accuracy = correct_predictions / total_sentences
        print(f'Evaluation Loss: {average_loss}, Accuracy: {accuracy}')

# Perform evaluation
evaluate(encoder, decoder, dataloader, criterion)


Epoch 0, Training Loss: 4.215738144787874
Epoch 10, Training Loss: 3.074706863899915
Epoch 20, Training Loss: 2.1177020613028077
Epoch 30, Training Loss: 0.9093303196763433
Epoch 40, Training Loss: 0.206132860509385
Epoch 50, Training Loss: 0.08091406935552942
Epoch 60, Training Loss: 0.049083839361967696
Epoch 70, Training Loss: 0.03435058035255953
Epoch 80, Training Loss: 0.02641932579309172
Epoch 90, Training Loss: 0.02131111375485295
Input String: Nous regardons des films le vendredi, Target String: We watch movies on Fridays, Predicted String: We watch movies on Fridays
Input String: Le soleil brille, Target String: The sun is shining, Predicted String: The sun is shining
Input String: Ils boivent du café le matin, Target String: They drink coffee in the morning, Predicted String: They drink coffee in the morning
Input String: J'ai froid, Target String: I am cold, Predicted String: I am cold
Evaluation Loss: 0.016923232646123377, Accuracy: 1.0
